## 📌 RegularSurface Description

A **RegularSurface** is described by:

- 📍 **Origin in UTM coordinates**: Defined as `xori` and `yori`.
- 📏 **Increment in each direction**: Defined as `xinc` and `yinc`.
- 🔢 **Grid dimensions**: Defined by `ncol` (columns in X direction) and `nrow` (rows in Y direction).
- 🌀 **Rotation**: Counter-clockwise from the X (East) axis, given in degrees.
- 🔄 **Y-axis flipping**:
  - Normally, the system is **left-handed** (Z-axis positive down).
  - If `yflip = -1`, then the map becomes **right-handed**.
- 🖥 **2D array of values**:
  - Stored as a **masked NumPy array** with shape `(ncol, nrow)`.
  - Undefined map nodes are **masked**.
  - Stored in **C-order** (row-major).
  - Default data type: **64-bit Float**.


![Regular Surface Model](https://xtgeo.readthedocs.io/en/latest/_images/datamodel_regsurface.svg)


In [1]:
import numpy as np
import xtgeo

# Grid dimensions
ncol, nrow = 30, 50

# Create dummy values for the surface (e.g., elevations)
vals = np.linspace(1000, 2000, num=ncol*nrow)

surf = xtgeo.RegularSurface(
    ncol=30, nrow=50, xori=1234.5, yori=4321.0, xinc=30.0, yinc=50.0,
    rotation=30.0, values=vals, yflip=1,
)

In [2]:
print(surf)

Description of RegularSurface instance
Object ID                                => 139713009457360
File source                              => None
Shape: NCOL, NROW                        => 30  50
Active cells vs total                    => 1500  1500
Origins XORI, YORI                       => 1234.5  4321.0
Increments XINC YINC                     => 30.0  50.0
Rotation (anti-clock from X)             => 30.0
YFLIP flag                               => 1
Inlines vector                           => [ 1  2  3 ... 28 29 30]
Xlines vector                            => [ 1  2  3 ... 48 49 50]
Values                                   => [1000.0 1000.667111407605 1001.3342228152102 ... 1998.66577718479
 1999.332888592395 2000.0]  float64
Values: mean, stdev, minimum, maximum    => 1500.0  288.86764887736075  1000.0  2000.0
Minimum memory usage of array (GB)       => 1.1175870895385742e-05


In [4]:
print(surf.values)

[[1000.0 1000.667111407605 1001.3342228152102 ... 1031.3542361574382
  1032.0213475650435 1032.6884589726485]
 [1033.3555703802535 1034.0226817878586 1034.6897931954636 ...
  1064.7098065376917 1065.3769179452968 1066.0440293529018]
 [1066.711140760507 1067.378252168112 1068.0453635757171 ...
  1098.0653769179453 1098.7324883255503 1099.3995997331554]
 ...
 [1900.6004002668446 1901.2675116744497 1901.9346230820547 ...
  1931.9546364242829 1932.621747831888 1933.288859239493]
 [1933.9559706470982 1934.623082054703 1935.2901934623083 ...
  1965.3102068045364 1965.9773182121414 1966.6444296197465]
 [1967.3115410273517 1967.9786524349565 1968.6457638425618 ...
  1998.66577718479 1999.332888592395 2000.0]]


In [5]:
surf.values.shape

(30, 50)

In [6]:
surf.values1d.shape

(1500,)

In [11]:
print('Number of data points in the surface data: ', len(surf.values1d))
print('Minimum value: ', np.min(surf.values1d))
print('Minimum value: ', np.min(surf.values))
print('Maximum value: ', np.max(surf.values))
print('Surface rotation: ', surf.rotation)

Number of data points in the surface data:  1500
Minimum value:  1000.0
Minimum value:  1000.0
Maximum value:  2000.0
Surface rotation:  30.0


In [1]:
from bokeh.plotting import figure, show, row
from bokeh.models import ColorBar, LinearColorMapper, ColumnDataSource
from bokeh.io import output_notebook

# Enable Bokeh for notebooks
output_notebook()

# Get X, Y coordinates and flatten them
x, y = surf.get_xy_values()
x = x.flatten()
y = y.flatten()
z = surf.values.flatten()

# Create a ColumnDataSource
source = ColumnDataSource(data=dict(x=x, y=y, z=z))

# Define color mapper
color_mapper = LinearColorMapper(palette="Viridis256", low=z.min(), high=z.max())

# Create Bokeh figure
p = figure(title="Surface Elevation", 
           tools='pan,wheel_zoom,reset,hover',
           tooltips=[("X", "@x"), ("Y", "@y"), ("Elevation", "@z")],
           match_aspect=True)

# Add scatter plot using ColumnDataSource
p.scatter(x='x', y='y', source=source, size=10, 
         color={'field': 'z', 'transform': color_mapper}, 
         line_color=None)


# Create Bokeh figure
p2 = figure(title="Surface Elevation", 
           tools='pan,wheel_zoom,reset,hover',
           tooltips=[("X", "@x"), ("Y", "@y"), ("Elevation", "@z")],
           match_aspect=True)


# Add scatter plot using ColumnDataSource
p2.scatter(x='x', y='y', source=source, size=5, 
         color={'field': 'z', 'transform': color_mapper}, 
         line_color=None)


# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, location=(0,0), title='Elevation')
p.add_layout(color_bar, 'left')

# Show the plot
show(row(p, p2))

ModuleNotFoundError: No module named 'bokeh'

In [8]:
surf.to_file("newfile.gri")

PosixPath('newfile.gri')

In [9]:
surf3 =xtgeo.surface_from_file('newfile.gri')

print(surf3)  # will show a description

print(surf3.xinc, surf3.yinc)

print(surf3.rotation)

# change the rotation:
surf3.rotation = 45.0

# move the surface 1000 m to west:
surf3.xori -= 1000.0

# export the modified surface
surf3.to_file('changedsurface.gri')  # irap binary is default

# Note that changing `nrow` and `ncol` is not possible to do directly.

Description of RegularSurface instance
Object ID                                => 139711672331280
File source                              => newfile.gri
Shape: NCOL, NROW                        => 30  50
Active cells vs total                    => 1500  1500
Origins XORI, YORI                       => 1234.5  4321.0
Increments XINC YINC                     => 30.0  50.0
Rotation (anti-clock from X)             => 30.0
YFLIP flag                               => 1
Inlines vector                           => [ 1  2  3 ... 28 29 30]
Xlines vector                            => [ 1  2  3 ... 48 49 50]
Values                                   => [1000.0 1000.6671142578125 1001.334228515625 ... 1998.665771484375
 1999.3328857421875 2000.0]  float64
Values: mean, stdev, minimum, maximum    => 1500.0  288.8676482416572  1000.0  2000.0
Minimum memory usage of array (GB)       => 1.1175870895385742e-05
30.0 50.0
30.0


PosixPath('changedsurface.gri')